In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import *
from pyspark.ml.regression import *
from pyspark.ml.evaluation import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import seaborn as sns

In [2]:
spark = SparkSession.builder.appName("Vu dep trai").config("spark.executor.memory","12g").getOrCreate()
# conf = pyspark.SparkConf().setMaster("spark://node-master:7077")\
#         .setAppName("Vu dep trai")\
#         .set("spark.executor.memory","15g")
# # sc = SparkContext.getOrCreate(conf=conf)
# # spark.stop()
# sc = SparkContext(conf = conf)
# spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/03 12:08:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_stores_raw = spark.read.csv("data/ba-walmart/stores.csv", header=True, inferSchema=True)
df_feature_raw = spark.read.csv("data/ba-walmart/features.csv", header=True, inferSchema=True)
df_train_raw = spark.read.csv("data/ba-walmart/train.csv", header=True, inferSchema=True)
df_test_raw = spark.read.csv("data/ba-walmart/test.csv", header=True, inferSchema=True)

In [4]:
df_feature = df_feature_raw.drop("MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5")
df = df_train_raw.join(df_feature, how="left", on=["Store", "Date", "IsHoliday"], ).join(df_stores_raw, how="left", on=["Store"])
df_test = df_test_raw.join(df_feature, how="left", on=["Store", "Date", "IsHoliday"]).join(df_stores_raw, how="left", on=["Store"])
df = df.withColumn("CPI", df["CPI"].cast(FloatType())).withColumn("Unemployment", df["Unemployment"].cast(FloatType()))
df_test = df_test.withColumn("CPI", df_test["CPI"].cast(FloatType())).withColumn("Unemployment", df_test["Unemployment"].cast(FloatType()))


In [5]:
df = df.withColumn("Year", year("Date")).withColumn("Month", month("Date")).withColumn("Week", weekofyear("Date"))
df_test = df_test.withColumn("Year", year("Date")).withColumn("Month", month("Date")).withColumn("Week", weekofyear("Date"))
df = df.withColumn("IsHoliday", df["IsHoliday"].cast(IntegerType()))
df_test = df_test.withColumn("IsHoliday", df_test["IsHoliday"].cast(IntegerType()))

In [6]:
df_clean = df.filter(df["Weekly_Sales"] > 0)
df_clean = df_clean.filter(df_clean["Weekly_Sales"] < 450000)

In [7]:
types = df_clean.select("Type").distinct().collect()
types.sort()
mapping = {t.Type: str(i) for i, t in enumerate(types)}
df_clean = df_clean.replace(mapping, subset=["Type"])
df_test = df_test.replace(mapping, subset=["Type"])
df_clean = df_clean.withColumn("Type", df_clean["Type"].cast(IntegerType()))
df_test = df_test.withColumn("Type", df_test["Type"].cast(IntegerType()))

In [8]:
## From EDA select important columns
input_col = ['Store', 'IsHoliday', 'Type', 'Size', 'Week','Dept','Year']
target = 'Weekly_Sales'

In [9]:
df_train, df_valid = df_clean.randomSplit([0.8, 0.2], seed=1234)

In [17]:
assembler = VectorAssembler(inputCols=input_col, outputCol="features")
rf = RandomForestRegressor(featuresCol="features", labelCol="Weekly_Sales", maxDepth=35, numTrees=20, seed=42, maxBins=100000)
pipeline = pyspark.ml.Pipeline(stages=[assembler, rf])
model = pipeline.fit(df_train)
# rf = RandomForestRegressor(featuresCol="features", labelCol=target)
# pipeline = pyspark.ml.Pipeline(stages=[assembler, rf])
# paramGrid = pyspark.ml.tuning.ParamGridBuilder()\
#     .addGrid(rf.numTrees, [10])\
#     .addGrid(rf.maxDepth, [20])\
#     .addGrid(rf.maxBins, [100])\
#     .addGrid(rf.featureSubsetStrategy, ["0.8"])\
#     .addGrid(rf.subsamplingRate, [0.2])\
#     .build()
# evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mse")
# crossval = pyspark.ml.tuning.CrossValidator(estimator=pipeline,
#                             estimatorParamMaps=paramGrid,
#                             evaluator=evaluator,
#                             numFolds=3)
# cvModel = crossval.fit(df_train)

IllegalArgumentException: RandomForestRegressor_167f86514a27 parameter maxDepth given invalid value 35.

In [ ]:
# bestModel = cvModel.bestModel
# bestModel.stages[-1].extractParamMap()

In [15]:
train_pred = model.transform(df_train)
valid_pred = model.transform(df_valid)
evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="mae")
train_mae = evaluator.evaluate(train_pred)
valid_mae = evaluator.evaluate(valid_pred)
print("Train MAE: ", train_mae)
print("Valid MAE: ", valid_mae)

Train MAE:  7732.152219107972
Valid MAE:  8610.979905817512
